<a href="https://colab.research.google.com/github/T2718/AI_Make_Image/blob/main/CivitAI3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Hugging Face

!pip install diffusers transformers accelerate safetensors

import os
import torch
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline
from PIL import Image
from safetensors.torch import load_file
import torch


model_list = ['stablediffusionapi/eleet-model',
              'stablediffusionapi/brav6',
              'Vsukiyaki/ShiratakiMix',
              'stabilityai/stable-diffusion-xl-base-1.0'
              ]

#https://huggingface.co/MODEL_ID/~/resolve/main/model_index.json
model_id = ''
while True:
  set = input('設定(未入力でOK)')
  if set == 'model':
    model_id = input('model_idを入力')
  elif set == 'model_list':
    for k in range(len(model_list)):
      print(str(1+k)+':'+model_list[k])
    model_id = model_list[int(input('何番を使用しますか？'))-1]

  elif set != '':
    print('設定が見つかりません。')
    continue
  else:
    break



if model_id == '':
  # Stable Diffusionのモデルをロード
  model_id = "stablediffusionapi/eleet-model"


# ユーザーからの入力を取得
prompt = input("プロンプトを入力してください (強さをつける場合は形式: 'prompt1:weight, prompt2:weight'): ")
negative_prompt = input("ネガティブプロンプトを入力してください: ")
num_images = int(input("生成する画像の枚数を入力してください: "))
num_inference_steps = int(input("推論ステップ数を入力してください: "))
mode = input("text to image または image to image を選んでください (t/i): ")

HugCiv = input('Hugging Face(H) or CivitAI(C): ')
if HugCiv == 'C':
  model_id = "./models/Stable-diffusion/convert_model"

!mkdir output

# フォルダ名をプロンプトの前15文字にする
output_dir = 'output/'+prompt[:15].replace(" ", "_")  # スペースをアンダースコアに置き換え
os.makedirs(output_dir, exist_ok=True)

# 画像生成
if mode.lower() == 't':
    pipe = StableDiffusionPipeline.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        safety_checker=None  # Safe checkを無効に
    )
    pipe = pipe.to("cuda")  # GPUが必要です
    k_count = 0
    original_output = output_dir
    while True:
        output_dir = original_output+'/'+str(k_count)
        os.makedirs(output_dir, exist_ok=True)
        for i in range(num_images):
            image = pipe(
                prompt=prompt,
                negative_prompt=negative_prompt,
                num_inference_steps=num_inference_steps,
                # Explicitly pass an empty dictionary for added_cond_kwargs
                added_cond_kwargs={}
            ).images[0]
            # 画像をファイルに保存 (数字のファイル名)
            image.save(os.path.join(output_dir, f"{i + 1}.png"))
        k_plus = input('追加数(しない場合は0): ')
        if(k_plus == '0'):
            break
        print(prompt)
        k_prompt = input('プロンプト(変更しない場合は無入力): ')
        print(negative_prompt)
        k_negative_prompt = input('ネガティブプロンプト(変更しない場合は無入力): ')
        if k_prompt != '':
            prompt = k_prompt
        if k_negative_prompt != '':
            negative_prompt = k_negative_prompt

        num_images = int(k_plus)
        k_count += 1

elif mode.lower() == 'i':
    image_path = input("元画像のpathを入力してください(例: img.png): ")
    strength = float(input("元画像の変更量(0~1): "))
    original_image = Image.open(image_path).convert("RGB")

    # Image-to-Imageパイプライン
    pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        safety_checker=None
    )
    pipe = pipe.to("cuda")


    for i in range(num_images):
        image = pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            image=original_image,
            strength=strength,  # 元画像をどれだけ変化させるか (0～1)
            num_inference_steps=num_inference_steps
        ).images[0]
        # 画像をファイルに保存 (数字のファイル名)
        image.save(os.path.join(output_dir, f"{i + 1}.png"))

print(f"生成された画像は '{output_dir}' フォルダに保存されました。")

In [1]:
#CivitAI

#「1」「1」「」を入力でいい


import os
import re

# --- 1. ダウンロード先のディレクトリ設定 ---
base_dir = "/content/models" # モデルとLoRAを同じディレクトリに保存する例
os.makedirs(base_dir, exist_ok=True)


# --- 2. ユーザーにダウンロードするタイプを選択させる ---
print("--- モデルダウンロードセレクター ---")
print("1. Stable Diffusion Checkpoint モデル (例: .safetensors, .ckpt)")
print("2. LoRA モデル (例: .safetensors)")
print("-----------------------------------")

download_choice = input("ダウンロードしたいタイプを数字で入力してください (1 または 2): ")

# --- 3. 選択に応じたダウンロード処理 ---

if download_choice == '1':
  # Checkpoint モデルのダウンロード
  print("\n--- Checkpoint モデルのダウンロード ---")
  print("CivitaiでダウンロードしたいCheckpointモデルのダウンロードリンクまたはIDをコピーし、ここに貼り付けてください。")
  print("例: https://civitai.com/api/download/models/12345または12345")
  model_url = input("CheckpointモデルのダウンロードURL: ")

  regex = r'([0-9]{5})'
  if bool(re.match(regex, model_url)):
    model_url = 'https://civitai.com/api/download/models/'+model_url
  elif model_url == '1':
    model_url = "https://civitai.com/api/download/models/28100?type=Model&format=SafeTensor&size=full&fp=fp16"
  elif model_url == '2':
    model_url = "https://civitai.com/api/download/models/11812?type=Model&format=SafeTensor&size=full&fp=fp16"


  # URLからファイル名を推測するか、ユーザーに入力させる
  # 通常、CivitaiのダウンロードURLはファイル名を含まないため、手動入力が確実
  default_filename = model_url.split('/')[-1] if '.' in model_url.split('/')[-1] else "downloaded_model.safetensors"
  model_filename = input(f"保存するファイル名を入力してください (デフォルト: {default_filename}): ")
  if not model_filename:
    model_filename = default_filename

  download_path = os.path.join(base_dir, model_filename)

  print(f"Downloading Checkpoint model to {download_path}...")
  !wget -O {download_path} "{model_url}"
  print(f"Download of {model_filename} complete!")

  # Checkpointモデルのパスを記録
  checkpoint_path = download_path
  lora_path = None # LoRAはダウンロードしないのでNone

elif download_choice == '2':
  # LoRA モデルのダウンロード
  print("\n--- LoRA モデルのダウンロード ---")
  print("CivitaiでダウンロードしたいLoRAモデルのダウンロードリンクをコピーし、ここに貼り付けてください。")
  print("例: https://civitai.com/api/download/models/98765")
  lora_url = input("LoRAモデルのダウンロードURL: ")

  # URLからファイル名を推測するか、ユーザーに入力させる
  default_filename = lora_url.split('/')[-1] if '.' in lora_url.split('/')[-1] else "downloaded_lora.safetensors"
  lora_filename = input(f"保存するファイル名を入力してください (デフォルト: {default_filename}): ")
  if not lora_filename:
    lora_filename = default_filename

  download_path = os.path.join(base_dir, lora_filename)

  print(f"Downloading LoRA model to {download_path}...")
  !wget -O {download_path} "{lora_url}"
  print(f"Download of {lora_filename} complete!")

  # LoRAモデルのパスを記録
  lora_path = download_path
  checkpoint_path = None # CheckpointはダウンロードしないのでNone

else:
  print("無効な選択です。1 または 2 を入力してください。")
  checkpoint_path = None
  lora_path = None

print("\n--- ダウンロード結果 ---")
if checkpoint_path:
  print(f"Checkpoint Path: {checkpoint_path}")
elif lora_path:
  print(f"LoRA Path: {lora_path}")
else:
  print("モデルはダウンロードされませんでした。")

# --- この後、diffusersのパイプラインにダウンロードしたパスを渡す ---
# 例:
# if checkpoint_path:
#   pipeline = AutoPipelineForText2Image.from_single_file(checkpoint_path, ...)
# if lora_path:
#   # base_model_path は別途ロードする必要がある
#   # pipeline.load_lora_weights(lora_path, ...)

--- モデルダウンロードセレクター ---
1. Stable Diffusion Checkpoint モデル (例: .safetensors, .ckpt)
2. LoRA モデル (例: .safetensors)
-----------------------------------
ダウンロードしたいタイプを数字で入力してください (1 または 2): 1

--- Checkpoint モデルのダウンロード ---
CivitaiでダウンロードしたいCheckpointモデルのダウンロードリンクをコピーし、ここに貼り付けてください。
例: https://civitai.com/api/download/models/12345
CheckpointモデルのダウンロードURL: https://civitai.com/api/download/models/11812?type=Model&format=SafeTensor&size=full&fp=fp16
保存するファイル名を入力してください (デフォルト: downloaded_model.safetensors): 
--2025-05-20 23:48:09--  https://civitai.com/api/download/models/11812?type=Model&format=SafeTensor&size=full&fp=fp16
Resolving civitai.com (civitai.com)... 172.67.12.143, 104.22.18.237, 104.22.19.237, ...
Connecting to civitai.com (civitai.com)|172.67.12.143|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: https://civitai-delivery-worker-prod.5ac0637cfd0766c97916cefa3764fbdf.r2.cloudflarestorage.com/104608/model/aom3a2.sB30.safetensors?X-Amz-Expire

In [2]:
!mkdir /content/models

!wget -O /content/models/v1-inference.yaml \
https://raw.githubusercontent.com/CompVis/stable-diffusion/main/configs/stable-diffusion/v1-inference.yaml

mkdir: cannot create directory ‘/content/models’: File exists
--2025-05-20 23:49:41--  https://raw.githubusercontent.com/CompVis/stable-diffusion/main/configs/stable-diffusion/v1-inference.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1873 (1.8K) [text/plain]
Saving to: ‘/content/models/v1-inference.yaml’

/content/models/v1- 100%[===================>]   1.83K  --.-KB/s    in 0s      

2025-05-20 23:49:41 (26.5 MB/s) - ‘/content/models/v1-inference.yaml’ saved [1873/1873]



In [3]:
!mkdir /content/converted
!wget -O convert_original_stable_diffusion_to_diffusers.py \
https://raw.githubusercontent.com/huggingface/diffusers/main/scripts/convert_original_stable_diffusion_to_diffusers.py
!python convert_original_stable_diffusion_to_diffusers.py \
  --checkpoint_path ./models/downloaded_model.safetensors \
  --original_config_file ./models/v1-inference.yaml \
  --dump_path ./converted/diffusers_model \
  --from_safetensors

--2025-05-20 23:49:41--  https://raw.githubusercontent.com/huggingface/diffusers/main/scripts/convert_original_stable_diffusion_to_diffusers.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7020 (6.9K) [text/plain]
Saving to: ‘convert_original_stable_diffusion_to_diffusers.py’

convert_original_st 100%[===================>]   6.86K  --.-KB/s    in 0s      

2025-05-20 23:49:42 (78.6 MB/s) - ‘convert_original_stable_diffusion_to_diffusers.py’ saved [7020/7020]

2025-05-20 23:49:57.712479: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747784997.734700    1527 cuda_dnn.cc:8310] Unable to register cuDNN factory: A

In [4]:
!pip install diffusers transformers accelerate safetensors


from diffusers import StableDiffusionPipeline
import torch
from PIL import Image
import os

pipe = StableDiffusionPipeline.from_pretrained(
    "/content/converted/diffusers_model",
    torch_dtype=torch.float16,
    use_safetensors=False,
    safety_checker=None
).to("cuda")

#prompt = "1girl, pastel colors, dreamy background"
#negative_prompt = "low quality, blurry"

prompt = input('prompt:')
negative_prompt = input('negative_prompt')
count = int(input('count:'))
strength = int(input('strength:'))

for k in range(count):
  image = pipe(prompt=prompt, negative_prompt=negative_prompt, num_inference_steps=strength).images[0]
  os.makedirs("./output/"+prompt[0:10], exist_ok=True)
  image.save("./output/"+prompt[0:10]+"/"+str(k)+".png")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


prompt:Anime, spread Legs, nude, bare,cat ears, on bed,Pouring a large amount of white sticky liquid over the body ,injection white liquid from crotch, big sized breasts, strong border, blush,shiny eyes, gold eyes, silver hair, braid hair
negative_promptreal, low quality
count:10
strength:100


  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
#Other

!pip install transformers

from transformers import CLIPTokenizer

tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")

prompt = input('prompt : ')
tokens = tokenizer(prompt)["input_ids"]

print("トークン数:", len(tokens))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-bd3f826e9d99>", line 7, in <cell line: 0>
    prompt = input('prompt : ')
             ^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 1177, in raw_input
    return self._input_request(
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 1219, in _input_request
    raise KeyboardInterrupt("Interrupted by user") from None
KeyboardInterrupt: Interrupted by user

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
          ^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'Keyb

TypeError: object of type 'NoneType' has no len()

In [ ]:
!rm -r output